In [58]:
import numpy as np
import sys
sys.path.append('E:/BJCWorkshop/LPF-Defense-main/model')
print(sys.path)
# Removed Google Colab specific code

# Example of setting a local path
# local_path = 'E:/BJCWorkshop/LPF-Defense-main/model/Data/'

# Use local_path in your code where you need to access files
# For example, loading a file:
# data = np.load(local_path + 'train_data_attack.npy')

['d:\\anaconda3\\envs\\pytorch_env\\python39.zip', 'd:\\anaconda3\\envs\\pytorch_env\\DLLs', 'd:\\anaconda3\\envs\\pytorch_env\\lib', 'd:\\anaconda3\\envs\\pytorch_env', '', 'd:\\anaconda3\\envs\\pytorch_env\\lib\\site-packages', 'd:\\anaconda3\\envs\\pytorch_env\\lib\\site-packages\\win32', 'd:\\anaconda3\\envs\\pytorch_env\\lib\\site-packages\\win32\\lib', 'd:\\anaconda3\\envs\\pytorch_env\\lib\\site-packages\\Pythonwin', 'E:/BJCWorkshop/LPF-Defense-main/model/models/', 'E:/BJCWorkshop/LPF-Defense-main/model/models/', 'E:/BJCWorkshop/LPF-Defense-main/model/models/', 'E:/BJCWorkshop/LPF-Defense-main/model/models/', 'E:/BJCWorkshop/LPF-Defense-main/model/models/', 'E:/BJCWorkshop/LPF-Defense-main/model/models/', 'E:/BJCWorkshop/LPF-Defense-main/model/models/', 'E:/BJCWorkshop/LPF-Defense-main/model/models/', 'E:/BJCWorkshop/LPF-Defense-main/model/models/', 'E:/BJCWorkshop/LPF-Defense-main/model/models/', 'E:/BJCWorkshop/LPF-Defense-main/model/models/', 'E:/BJCWorkshop/LPF-Defense-main/

In [59]:
# %cd '/content/drive/MyDrive/Kimia'

In [60]:
import os
import sys
import torch
import numpy as np
import importlib
from torchvision import transforms
from data_utils.ModelNetDataLoader import ModelNetDataLoader, load_data, PointSampler, Normalize, RandomNoise, RandomRotate, toTensor
import importlib.util


In [61]:
if torch.cuda.is_available():
      device = torch.device('cuda:0')
      print('running on GPU')
else:
      device = torch.device('cpu')
      print('running on CPU')

running on GPU


# Dynamic Saliency Map 

**Please specify the path to the models based on your setup.**

For pointnet and pointnet2 run the following cells

In [74]:
# import os
# import sys
# import importlib
# import torch
# import numpy as np
# # import model.train_classification as train_classification
# # import model.eval_classification as eval_classification

# # Add the directory containing pointnet to the Python path
# # sys.path.append('E:/BJCWorkshop/LPF-Defense-main/model/models/')
# sys.path.append('E:/BJCWorkshop/LPF-Defense-main/model/log/classification')

# # Set experiment directory
# experiment_dir = 'E:/BJCWorkshop/LPF-Defense-main/model/log/classification/001_model'

# # Set experiment directory
# # log_dir = '001_model'
# # experiment_dir = 'E:/BJCWorkshop/LPF-Defense-main/model/log/classification/' + log_dir

# # num_class = 40

# # List all Python files in the logs directory
# log_files = [f for f in os.listdir(experiment_dir) if f.endswith('.py')]
# if not log_files:
#     raise FileNotFoundError("No Python files found in the logs directory.")

# print(log_files)

# # Load the module from the file path
# num_class = 40
# model_name = log_files[0].split('.')[0]

# # model_file_path = os.path.join(experiment_dir, model_name)
# # spec = importlib.util.spec_from_file_location("pointnet2_cls_ssg", model_file_path)
# # model_module = importlib.util.module_from_spec(spec)
# # sys.modules["pointnet2_cls_ssg"] = model_module
# # spec.loader.exec_module(model_module)


# model_module = importlib.import_module(model_name)
# print(model_module)
# classifier = model_module.get_model(num_class, normal_channel=False)

# # Load model weights
# checkpoint = torch.load(os.path.join(experiment_dir, 'checkpoints', 'best_model.pth'))
# classifier.load_state_dict(checkpoint['model_state_dict'])

# # Set device
# device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
# classifier.to(device)

# # Example operation: Evaluate model
# classifier.eval()
# with torch.no_grad():
#     # Assuming data is preprocessed and ready for input
#     local_path = 'E:/BJCWorkshop/LPF-Defense-main/model/Data/'
#     data = np.load(os.path.join(local_path, 'train_data_attack.npy'))
#     input_data = torch.from_numpy(data).float().to(device)
#     output = classifier(input_data)
#     print(output)



# Clear sys.path of unnecessary paths
sys.path = [p for p in sys.path if 'attacks' not in p]
print(sys.path)

# Add the correct directory to sys.path
sys.path.append('E:/BJCWorkshop/LPF-Defense-main/model/log/classification/001_model')
experiment_dir = 'E:/BJCWorkshop/LPF-Defense-main/model/log/classification/001_model'

num_class = 40

# List all Python files in the logs directory
log_files = [f for f in os.listdir(experiment_dir) if f.endswith('.py')]
if not log_files:
    raise FileNotFoundError("No Python files found in the logs directory.")

print(log_files)

# Load the module from the file path
num_class = 40
model_name = log_files[0].split('.')[0]
model_module = importlib.import_module(model_name)
print(model_module)
# Path to the model file
# model_file_path = 'E:/BJCWorkshop/LPF-Defense-main/model/log/classification/001_model/pointnet2_cls_ssg.py'

# Load the module from the file path
# spec = importlib.util.spec_from_file_location("pointnet2_cls_ssg", model_file_path)
# model_module = importlib.util.module_from_spec(spec)
# sys.modules["pointnet2_cls_ssg"] = model_module
# spec.loader.exec_module(model_module)
# print(model_module)
criterion = model_module.get_loss()
# Use the loaded module
classifier = model_module.get_model(num_class=40, normal_channel=False)

# Load model weights

checkpoint = torch.load(os.path.join(experiment_dir, 'checkpoints', 'best_model.pth'))
classifier.load_state_dict(checkpoint['model_state_dict'])

# Set device
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
classifier.to(device)

# # Example operation: Evaluate model
# classifier.eval()
# with torch.no_grad():
#     for data in testDataLoader:
#         points, labels = data
#         points = points.to(device)
#         labels = labels.to(device)
#         output = classifier(points)
#         print(output)
#         break  # Remove this break to process the entire dataset



['d:\\anaconda3\\envs\\pytorch_env\\python39.zip', 'd:\\anaconda3\\envs\\pytorch_env\\DLLs', 'd:\\anaconda3\\envs\\pytorch_env\\lib', 'd:\\anaconda3\\envs\\pytorch_env', '', 'd:\\anaconda3\\envs\\pytorch_env\\lib\\site-packages', 'd:\\anaconda3\\envs\\pytorch_env\\lib\\site-packages\\win32', 'd:\\anaconda3\\envs\\pytorch_env\\lib\\site-packages\\win32\\lib', 'd:\\anaconda3\\envs\\pytorch_env\\lib\\site-packages\\Pythonwin', 'E:/BJCWorkshop/LPF-Defense-main/model/models/', 'E:/BJCWorkshop/LPF-Defense-main/model/models/', 'E:/BJCWorkshop/LPF-Defense-main/model/models/', 'E:/BJCWorkshop/LPF-Defense-main/model/models/', 'E:/BJCWorkshop/LPF-Defense-main/model/models/', 'E:/BJCWorkshop/LPF-Defense-main/model/models/', 'E:/BJCWorkshop/LPF-Defense-main/model/models/', 'E:/BJCWorkshop/LPF-Defense-main/model/models/', 'E:/BJCWorkshop/LPF-Defense-main/model/models/', 'E:/BJCWorkshop/LPF-Defense-main/model/models/', 'E:/BJCWorkshop/LPF-Defense-main/model/models/', 'E:/BJCWorkshop/LPF-Defense-main/

C:\Users\Administrator\AppData\Local\Temp\ipykernel_26100\3065997685.py:101: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(os.path.join(experiment_di

get_model(
  (sa1): PointNetSetAbstraction(
    (mlp_convs): ModuleList(
      (0): Conv2d(3, 64, kernel_size=(1, 1), stride=(1, 1))
      (1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
      (2): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1))
    )
    (mlp_bns): ModuleList(
      (0-1): 2 x BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (sa2): PointNetSetAbstraction(
    (mlp_convs): ModuleList(
      (0): Conv2d(131, 128, kernel_size=(1, 1), stride=(1, 1))
      (1): Conv2d(128, 128, kernel_size=(1, 1), stride=(1, 1))
      (2): Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1))
    )
    (mlp_bns): ModuleList(
      (0-1): 2 x BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (sa3): PointNetSetAbstract

For DGCNN run the following cell

In [84]:
# DGCNN
sys.path.append('E:/BJCWorkshop/LPF-Defense-main/attacks/')
from DGCNN_cls import DGCNN, cal_loss
# k = 20
# emb_dims = 1024
# dropout_p = 0.5
# model = DGCNN(k, emb_dims, dropout_p, output_channels=40).to('cuda:0')
# checkpoint = torch.load('log/DGCNN_checkpoint.pt')
# model.load_state_dict(checkpoint['model_state_dict'])

d:\anaconda3\envs\pytorch_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


To load the data, run the following cell. Specify paths based on your setup.

In [85]:
main_path = 'E:/BJCWorkshop/LPF-Defense-main/model'
test_data_path = 'Data/test_data.npy'
test_label_path = 'Data/test_labels.npy'
test_X, test_y = load_data(main_path, test_data_path, test_label_path, mode='test')
test_X, test_y = test_X.astype(np.float32), test_y.astype(np.int64)
batch_size = test_X.shape[0]
default_transform = transforms.Compose([Normalize(), toTensor()])

test_dataset = ModelNetDataLoader(test_X, test_y, transform=default_transform)
testDataLoader = torch.utils.data.DataLoader(test_dataset, batch_size=16, shuffle=False, num_workers=10)
for data in testDataLoader:
  break
points, labels = data
print(points.shape)
print(labels.shape)

torch.Size([16, 1024, 3])
torch.Size([16])


In [92]:
def dynamic_saliency_map(model, points, labels, device, n=100, T=20, alpha=1):
    model.eval()
    saliency_points = np.zeros((points.shape[0], 3, n))
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    points = points.transpose(1, 2).float()  # B * 3 * 1024
    a = int(n/T)
    for i in range(T):
        points = points.to(device)
        labels = labels.to(device)
        # computing the gradient of the loss with respect to the points
        points.requires_grad = True
        
        # pointnet, pointnet++
        # **************************
        pred, trans_feat = classifier(points)
        loss = criterion(pred, labels.long(), trans_feat)
        # **************************
        
        # dgcnn
        # **************************
        # pred,_ = model(points)
        # loss = cal_loss(pred, labels.long())
        # ***************************
         # Check if loss requires grad
        if loss.requires_grad:
            loss.backward()
            grad = points.grad
            # ... rest of the code ...
        else:
            print("Loss does not require grad")
            return None, None
        # loss.backward()
        # grad = points.grad
        # # saliency map calculations
        sphere_core = torch.median(points, axis=2, keepdims=True)  # B * 3 * 1

        sphere_r = torch.sqrt(torch.sum(torch.square(
            points - sphere_core.values), axis=1))  # B * 1024

        sphere_axis = points - sphere_core.values  # B * 3 * 1024

        before_map = torch.sum(torch.multiply(
            grad, sphere_axis), axis=1)  # B * 1024

        map = -torch.multiply(before_map,
                              torch.pow(sphere_r, alpha))  # B * 1024

        drop_indices = torch.topk(map, a).indices  # B * n/T

        tmp = torch.zeros((points.shape[0], 3, points.shape[2] - a))
        numpy_points = points.cpu().detach().numpy()
        numpy_indices = drop_indices.cpu().detach().numpy()
        for j in range(points.shape[0]):
            saliency_points[j][:, i *
                               a: (i + 1) * a] = numpy_points[j][:, numpy_indices[j]]
            tmp[j] = torch.from_numpy(
                np.delete(numpy_points[j], numpy_indices[j], axis=1))

        points = tmp.detach().clone()

    # returns the points after dropping the most important ones and the most important points
    return points, saliency_points

In [93]:
# Ensure the model is on the correct device
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
classifier.to(device)

# Evaluate model
classifier.eval()
with torch.no_grad():
    for data in testDataLoader:
        points, labels = data
        points = points.to(device)  # Move points to the same device as the model
        labels = labels.to(device)  # Move labels to the same device as the model

        # Print shapes for debugging
        print(points.shape)
        print(labels.shape)

        # Call the dynamic saliency map function
        new_points, saliency_points = dynamic_saliency_map(
            model=classifier, points=points, labels=labels, device=device, n=200, T=40
        )
        
        # Process the output as needed
        print(new_points)
        break  # Remove this break to process the entire dataset
    
i = False
j = 0
for data in testDataLoader:
    points, labels = data
    new_points, saliency_points = dynamic_saliency_map(model=model, points=points, labels=labels, device=device, n = 200, T = 40)
    if i is False:
        final_set = new_points.cpu().detach().numpy()
        i = True
    else:
        final_set = np.append(final_set, new_points, axis = 0)
    print(j)
    j+=1

torch.Size([16, 1024, 3])
torch.Size([16])
Loss does not require grad
None
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154


In [100]:
# import matplotlib.pyplot as plt
# from mpl_toolkits.mplot3d import Axes3D

# # 假设new_points是三维点云数据
# fig = plt.figure()
# ax = fig.add_subplot(111, projection='3d')
# ax.scatter(new_points[:, 0], new_points[:, 1], new_points[:, 2])
# plt.show()
# to_save = final_set.transpose(0, 2, 1)
# np.save('Data/drop200_pointnet2/test_data.npy', to_save)

#Ensure the directory exists
output_dir = 'Data/drop200_pointnet2'
os.makedirs(output_dir, exist_ok=True)

# Save the file
to_save = final_set.transpose(0, 2, 1)
np.save(os.path.join(output_dir, 'test_data.npy'), to_save)